# Metropolis-Hastings

In [155]:
#from __future__ import print_function
import numpy as np
import math

#import matplotlib
#import matplotlib.mlab as mlab

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go

import scipy.io as sio
import pandas as pd

from scipy import optimize, linalg
from pprint import pprint
from jupyterthemes import jtplot

# oceans16 | grade3 | chesterish | onedork | monokai | solarizedl
# set "context" (paper, notebook, talk, or poster)
# & font scale (scalar applied to labels, legend, etc.)
jtplot.style('grade3', context='paper', fscale=1.4)
jtplot.style(ticks=True, grid=False)
jtplot.figsize(x=15, y=10)

In [54]:
def laplacecomp(w, X, t):
    w = np.array(w)
    X = np.array(X)
    t = np.array(t)
    '''
        Return two np array for g and log(g)
    '''
    # Computes g and log g for the laplace model introduced in the lecture.
    ss = 10 # Prior variance
    # Evaluate log prior
    logg = -(1/(2 * ss)) * w.transpose().dot(w)
    # Compute P
    P = np.array(1. / ( 1 + np.exp(-X.dot(w))))
    #print(P)
    logl = sum((t * np.log(P)) + ((1 - t) * np.log(1-P)))
    
    logg = logg + logl
    g = np.exp(logg)
    return (g[0][0], logg[0][0])

w = np.array([[-1],[-2]])
X = np.matrix([[0, 0], [1,1]])
t = np.array([0,1])
assert laplacecomp(w, X, t) == (0.37093273795143539, -0.99173453213368734), 'Error in laplacomp function'

In [114]:
mat = sio.loadmat('binaryclass2.mat')
X = mat['X']
t = mat['t']
w = np.zeros([X.shape[1], 1])
#laplacecomp(w, X, t)

In [169]:
df = pd.DataFrame(X)
df0 = pd.DataFrame(t)
df.insert(2, 2, value=df0)

In [176]:
trace1 = go.Scatter(
    x = df[df[2] == 0][0],
    y = df[df[2] == 0][1],
    mode = 'markers',
    name = 'Class 0'
)

trace2 = go.Scatter(
    x = df[df[2] == 1][0],
    y = df[df[2] == 1][1],
    mode = 'markers',
    name = 'Class 1'
)

data = [trace1, trace2]

py.iplot(data, filename='basic-scatter')

In [56]:
def acceptance_ratio(w_s, w_s1, X, t):
    #print(w_s)
    r = laplacecomp(w_s, X, t)[1] - laplacecomp(w_s1, X, t)[1]
    #print(laplacecomp(w_s, X, t))
    return r

w = np.zeros([X.shape[1], 1])

w = np.random.multivariate_normal(mean=np.ones([X.shape[1], 1])[:,0], cov=step)
w = np.array([w]).T

new_w_s = np.random.multivariate_normal(mean=w[:,0], cov=step)
new_w_s = np.array([new_w_s]).T

print(laplacecomp(new_w_s, X, t))
print(acceptance_ratio(new_w_s, w, X, t))

In [259]:
def MH(X, t, total_trials, step):
    s = 0
    w = np.random.randn(2,1)
    ws = [w, ] #<---
    #w.append(np.ones([X.shape[1], 1]))
    for s in np.arange(total_trials):
        #s += 1
        if (s+1) % (total_trials/100) == 0:
            if (s+1) % (total_trials/10) == 0:
                print('|', end='')
            else:
                print('.', end='')
                
        wp = np.random.randn(2,1) * 0.5 + w #generate wp from p(ws|ws-1)
        w = ws[-1]
        gws, loggws = laplacecomp(w, X, t)
        gwp, loggwp = laplacecomp(wp, X, t)
        #compute acceptance ratio r
        logr = loggwp - loggws
        r = np.exp(logr)
    
        if logr >= 0:
            ws.append(wp) #acceptance
        else:
            u = np.random.uniform(0,1)
            if u <= r:
                ws.append(wp) #acceptance
            else:
                ws.append(w) #rejection -> don't forget to add again the same value w
    print()
    return ws

In [262]:
step = np.array([[0.5, 0], [0, 0.5]])
num_trials = 1000
samples = MH(X, t, num_trials, step)
#pprint(samples[0:10])

.........|.........|.........|.........|.........|.........|.........|.........|.........|.........|


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning:

invalid value encountered in multiply



In [263]:
#print(np.array(samples)[:,0][:,0])
w_df = pd.DataFrame([np.array(samples)[:,0][:,0], np.array(samples)[:,1][:,0]]).transpose()
trace = go.Scattergl(
    x = w_df[0],
    y = w_df[1],
    mode = 'markers',
    name = 'Weights'
)
data = [trace]

py.iplot(data, filename='basic-scatter')

PlotlyRequestError: Hey there! You've hit one of our API request limits. 

To get unlimited API calls(10,000/day), please upgrade to a paid plan. 

UPGRADE HERE: https://goo.gl/i7glmM 

Thanks for using Plotly! Happy Plotting!

In [208]:
def predict(samples, x_new):
    p = 0.
    for w in samples:
        p += 1 / (1 + np.exp(-w.T.dot(x_new)))
    p /= len(samples)
    return p[0]

In [209]:
x_new = np.array([2,-4])
predict(samples, x_new)

0.038492388262846462

In [254]:
xmin, xmax, xstep = (-6, 6.5, 0.5)
ymin, ymax, ystep = (-6, 6.5, 0.5)

p = []
for y in np.arange(ymin, ymax, ystep):
    p_x = []
    for x in np.arange(xmin, xmax, xstep):
        x_new = np.array([x, y])
        p_x.append(predict(samples, x_new))
    p.append(p_x)

In [255]:
prob = [go.Contour(
            z=p,
            y=np.arange(xmin, xmax, xstep),
            x=np.arange(ymin, ymax, ystep),
            contours=dict(coloring='heatmap')
    ), go.Scatter(
        x = df[df[2] == 0][0],
        y = df[df[2] == 0][1],
        mode = 'markers',
        marker = dict(color='orange'),
        name = 'Class 0'
    ), go.Scatter(
        x = df[df[2] == 1][0],
        y = df[df[2] == 1][1],
        mode = 'markers',
        marker = dict(color='blue'),
        name = 'Class 1'
)]

py.iplot(prob, filename='contour-scatter')